In [ ]:
from IPython.display import display, HTML
import pandas as pd
from os import listdir
from os.path import isfile, join
from pprint import pprint
from tqdm import tqdm
import json
import io

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.font_manager import FontProperties
import numpy as np
from scipy.spatial.distance import euclidean, pdist, squareform, cosine

import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


sns.set(style="ticks")
plt.rcParams['axes.facecolor']='white'
task_order = ['Length',  'WordContent', 'Depth', 'TopConstituents', 'SubjNumber', 'ObjNumber', 'BigramShift', 'CoordinationInversion', 'OddManOut', 'Tense']
dict_task = {0:'Length',  1:'WordContent', 2:'Depth', 3:'TopConstituents', 4:'BigramShift', 5:'Tense', 6:'SubjNumber', 7:'ObjNumber', 8:'OddManOut', 9:'CoordinationInversion'}

In [ ]:
def get_results(dir_path='./mlp_results'):
    columns = ['data_path', 'cache_path', 'result_path', 'batch_size', 'cbatch_size', 'nhid', 'optim', 'kfold', 'tenacity', 'usepytorch', 'epoch_size', 'device']
    filenames = [f for f in listdir(dir_path) if isfile(join(dir_path, f)) if '.json' in f]
    list_result = []
    for filename in filenames:
        with open(join(dir_path, filename), 'r') as infile:
#             print(filename)
            results = json.load(infile)
            for key, result in results.items():
                list_result.append(result)
                
    df = pd.DataFrame(list_result)[['acc', 'head', 'layer', 'task', 'model_name', 'location', 'devacc']]
    
    for column in columns:
        try:
            df = df.drop(columns=column)
        except:
            pass

    return df

# Layer-Task Peformance Chart

In [ ]:
df1 = get_results(dir_path='./fine_mlp_results')
df1['classifier'] = 'fine'
df2 = get_results(dir_path='./mlp_results')
df2['classifier'] = 'coarse'
df1 = df1.append(df2)

df1 = df1.loc[df1['head'] == -1]
df1.loc[df1['model_name'] == 'bert-base-uncased', 'layer'] *= 100 / 12
df1.loc[df1['model_name'] == 'bert-large-uncased', 'layer'] *= 100 / 24


# df = df.loc[df['model_name'] == 'bert-base-uncased']
g = sns.relplot(x='layer', y="acc", col='task', col_wrap=5, style='classifier', col_order=task_order, hue='model_name', kind="line", legend="full", data=df1, facet_kws={'sharey':False, 'sharex':False}) #col_wrap=3
for ax in g.axes.flatten():
    ax.grid(True)
#     ax.set_ylim(0, 100)

In [ ]:
df = get_results(dir_path='./linear_head_wise_results')
large_bert = df.loc[df['model_name'] == 'bert-large-uncased']
# large_bert = large_bert.loc[large_bert['head'] == 2]
large_bert = large_bert[['acc', 'head', 'layer', 'task', 'model_name']]
# display(large_bert)
# large_bert = large_bert.pivot(index='layer', columns='task', values=['acc'])
g = sns.relplot(x='layer', y="acc", col='task', col_wrap=5, col_order=task_order, hue='head', kind="line", legend="full", data=large_bert, facet_kws={'sharey':False, 'sharex':False}) #col_wrap=3


# Layer-Task Performance Table

In [ ]:
df = get_results(dir_path='./linear_head_wise_results')
large_bert = df.loc[df['model_name'] == 'bert-base-uncased']
large_bert = large_bert.loc[large_bert['head'] == 2]
large_bert = large_bert.pivot(index='layer', columns='task', values=['acc'])


large_bert.columns = large_bert.columns.droplevel()
large_bert = large_bert[['Length',  'WordContent', 'Depth', 'TopConstituents', 'BigramShift', 'Tense', 'SubjNumber', 'ObjNumber', 'OddManOut', 'CoordinationInversion']]
large_bert

## Layer-wise Downstream Task Performance

In [ ]:
df = get_results(dir_path='./downstream_results')
df['pearson'] = df['pearson'] * 100
# df = df.loc[df['task'] != 'SICKEntailment']
# df = df.loc[df['task'] == 'STS12']

# df = df.loc['task']
g = sns.relplot(x='layer', y="pearson", row='task', hue='head', kind="line", legend="full", data=df, facet_kws={'sharey':False, 'sharex':False}) #col_wrap=3
for ax in g.axes.flatten():
    ax.grid(True)
#     ax.set_ylim(0, 100)
    ax.set_title('')



# Head Wise Data Inspection

In [ ]:
df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'bert-base-uncased']
df = df.loc[df['head'] >= 0]
df = df.loc[df['task'] == dict_task[1]] # Choose task
df = df.pivot(index='layer', columns='head', values=['acc'])
display(df)
sns.heatmap(df, cmap='Reds')

# Head Wise Sorted Data Inspection  

위 heatmap을 통해서는 정확하게 얼마나 분포하고 있는지 알기가 힘들기 때문에 sorted 시켜보았다.

In [ ]:
plt_cmap = 'plasma'# 'viridis'
subtask_order = ['Length', 'Depth', 'SubjNumber', 'CoordinationInversion', 'BigramShift', 'OddManOut']


def draw_heatmap(*args, **kwargs):
    data = kwargs.pop('data')
    d = data.pivot(index=args[1], columns=args[0], values=args[2])
    data = d.values
    data.sort(axis=1)
    vmin = np.min(data)
    vmax = np.max(data)
    vmin_ = vmin + 0.3*(vmax-vmin)
    vmax_ = vmax - 0.05*(vmax-vmin) 
    
    d.iloc[:,:] = np.flip(data, 1)
    sns.heatmap(d[::-1], **kwargs, vmin=vmin_, vmax=vmax_)

df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'bert-base-uncased']
df = df.loc[df['head'] >= 0]
fg = sns.FacetGrid(df, col='task', col_wrap=10, col_order=subtask_order)
fg.map_dataframe(draw_heatmap, 'head', 'layer', 'acc', cbar=True, square = False, cmap=plt_cmap)
fg.savefig('imgs/fig3_bertbase.svg', format='svg', bbox_inches='tight')

df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'bert-large-uncased']
df = df.loc[df['head'] >= 0]
fg = sns.FacetGrid(df, col='task', col_wrap=10, col_order=subtask_order)
fg.map_dataframe(draw_heatmap, 'head', 'layer', 'acc', cbar=True, square = False, cmap=plt_cmap)
fg.savefig('imgs/fig3_bertlarge.svg', format='svg', bbox_inches='tight')

df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'openai-gpt']
df = df.loc[df['head'] >= 0]
fg = sns.FacetGrid(df, col='task', col_wrap=10, col_order=subtask_order)
fg.map_dataframe(draw_heatmap, 'head', 'layer', 'acc', cbar=True, square = False, cmap=plt_cmap)
fg.savefig('imgs/fig3_gpt.svg', format='svg')

df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'gpt2']
df = df.loc[df['head'] >= 0]
fg = sns.FacetGrid(df, col='task', col_wrap=10, col_order=subtask_order)
fg.map_dataframe(draw_heatmap, 'head', 'layer', 'acc', cbar=True, square = False, cmap=plt_cmap)
fg.savefig('imgs/fig3_gpt2.svg', format='svg', bbox_inches='tight')


In [ ]:
plt_cmap = 'plasma'# 'viridis'

def draw_heatmap(*args, **kwargs):
    data = kwargs.pop('data')
    d = data.pivot(index=args[1], columns=args[0], values=args[2])
    data = d.values
    data.sort(axis=1)
    vmin = np.min(data)
    vmax = np.max(data)
    vmin_ = vmin + 0.3*(vmax-vmin)
    vmax_ = vmax - 0.05*(vmax-vmin) 
    
    d.iloc[:,:] = np.flip(data, 1)
    sns.heatmap(d[::-1], **kwargs, vmin=vmin_, vmax=vmax_)

df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'bert-base-uncased']
df = df.loc[df['head'] >= 0]
fg = sns.FacetGrid(df, col='task', col_wrap=3, col_order=task_order)
fg.map_dataframe(draw_heatmap, 'head', 'layer', 'acc', cbar=True, square = False, cmap=plt_cmap)
fg.fig.suptitle('BERT Base', y=1.02) 
fg.savefig('imgs/A1_bertbase.svg', format='svg', bbox_inches='tight')

df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'bert-large-uncased']
df = df.loc[df['head'] >= 0]
fg = sns.FacetGrid(df, col='task', col_wrap=3, col_order=task_order)
fg.map_dataframe(draw_heatmap, 'head', 'layer', 'acc', cbar=True, square = False, cmap=plt_cmap)
fg.fig.suptitle('BERT Large', y=1.02) 
fg.savefig('imgs/A1_bertlarge.svg', format='svg', bbox_inches='tight')

df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'openai-gpt']
df = df.loc[df['head'] >= 0]
fg = sns.FacetGrid(df, col='task', col_wrap=3, col_order=task_order)
fg.map_dataframe(draw_heatmap, 'head', 'layer', 'acc', cbar=True, square = False, cmap=plt_cmap)
fg.fig.suptitle('GPT', y=1.02) 
fg.savefig('imgs/A1_gpt.svg', format='svg')

df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'gpt2']
df = df.loc[df['head'] >= 0]
fg = sns.FacetGrid(df, col='task', col_wrap=3, col_order=task_order)
fg.map_dataframe(draw_heatmap, 'head', 'layer', 'acc', cbar=True, square = False, cmap=plt_cmap)
fg.fig.suptitle('GPT2', y=1.02) 
fg.savefig('imgs/A_gpt2.svg', format='svg', bbox_inches='tight')


In [ ]:
plt_cmap = 'plasma'# 'viridis'

def draw_heatmap(*args, **kwargs):
    data = kwargs.pop('data')
    d = data.pivot(index=args[1], columns=args[0], values=args[2])
    data = d.values
    #data.sort(axis=1)
    vmin = np.min(data)
    vmax = np.max(data)
    vmin_ = vmin + 0.3*(vmax-vmin)
    vmax_ = vmax - 0.05*(vmax-vmin) 
    
    d.iloc[:,:] = np.flip(data, 1)
    sns.heatmap(d[::-1], **kwargs, vmin=vmin_, vmax=vmax_)

df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'bert-base-uncased']
df = df.loc[df['head'] >= 0]
fg = sns.FacetGrid(df, col='task', col_wrap=3, col_order=task_order)
fg.map_dataframe(draw_heatmap, 'head', 'layer', 'acc', cbar=True, square = False, cmap=plt_cmap)
fg.fig.suptitle('BERT Base', y=1.02) 
fg.savefig('imgs/A2_unsort_bertbase.svg', format='svg', bbox_inches='tight')

df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'bert-large-uncased']
df = df.loc[df['head'] >= 0]
fg = sns.FacetGrid(df, col='task', col_wrap=3, col_order=task_order)
fg.map_dataframe(draw_heatmap, 'head', 'layer', 'acc', cbar=True, square = False, cmap=plt_cmap)
fg.fig.suptitle('BERT Large', y=1.02) 
fg.savefig('imgs/A2_unsort_bertlarge.svg', format='svg', bbox_inches='tight')

df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'openai-gpt']
df = df.loc[df['head'] >= 0]
fg = sns.FacetGrid(df, col='task', col_wrap=3, col_order=task_order)
fg.map_dataframe(draw_heatmap, 'head', 'layer', 'acc', cbar=True, square = False, cmap=plt_cmap)
fg.fig.suptitle('GPT', y=1.02) 
fg.savefig('imgs/A2_unsort_gpt.svg', format='svg')

df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'gpt2']
df = df.loc[df['head'] >= 0]
fg = sns.FacetGrid(df, col='task', col_wrap=3, col_order=task_order)
fg.map_dataframe(draw_heatmap, 'head', 'layer', 'acc', cbar=True, square = False, cmap=plt_cmap)
fg.fig.suptitle('GPT2', y=1.02) 
fg.savefig('imgs/A2_unsort_gpt2.svg', format='svg', bbox_inches='tight')


# Head-Wise Task Pair-plot

In [ ]:
df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'bert-base-uncased']
df = df.loc[df['head'] >= 0]
df = df.pivot_table(index=['layer', 'head'], columns='task', values='acc')
df = df.reset_index()
sns.pairplot(df, hue='layer', vars=['Length',  'WordContent', 'Depth', 'TopConstituents', 'BigramShift', 'Tense', 'SubjNumber', 'ObjNumber', 'OddManOut', 'CoordinationInversion'])

df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'bert-large-uncased']
df = df.loc[df['head'] >= 0]
df = df.pivot_table(index=['layer', 'head'], columns='task', values='acc')
df = df.reset_index()
sns.pairplot(df, hue='layer', vars=['Length',  'WordContent', 'Depth', 'TopConstituents', 'BigramShift', 'Tense', 'SubjNumber', 'ObjNumber', 'OddManOut', 'CoordinationInversion'])

# Task-Wise Clustering

In [ ]:
df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'bert-base-uncased']
df = df.loc[df['location'] == 'head']
df = df.loc[df['head'] >= 0]
    
df2 = get_results(dir_path='./ds_linear_head_wise_results')
df2 = df2.loc[df2['location'] == 'head']
df2 = df2.loc[df2['model_name'] == 'bert-base-uncased']
df2 = df2.loc[df2['head'] >= 0]
df = pd.concat([df, df2])
df = df.loc[df['task'] != 'MRPC']

df = df.pivot_table(index=['layer', 'head'], columns='task', values='acc')
df = df.reset_index().drop(columns=['head', 'layer'])
#df = df[task_order]

df = df.corr(method='spearman')

#sns.heatmap(df, cmap='bwr', center=0, robust=True)
g = sns.clustermap(df, cmap='bwr', figsize=(7,7), center=0)
#g.fig.suptitle('BERT Base') 
g.savefig('imgs/fig5_correlation.svg', format='svg', bbox_inches='tight')

In [ ]:
df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'bert-base-uncased']
df = df.loc[df['head'] >= 0]

df2 = get_results(dir_path='./ds_linear_head_wise_results')
df2 = df2.loc[df2['location'] == 'head']
df2 = df2.loc[df2['model_name'] == 'bert-base-uncased']
df2 = df2.loc[df2['head'] >= 0]
df = pd.concat([df, df2])
df = df.loc[df['task'] != 'MRPC']

df = df.pivot_table(index=['layer', 'head'], columns='task', values='acc')
df = df.reset_index().drop(columns=['head', 'layer'])
#df = df[task_order]

df = df.corr(method='spearman')

#sns.heatmap(df, annot=True, cmap='bwr', center=0)
g = sns.clustermap(df, cmap='bwr', figsize=(7,7), center=0)
g.fig.suptitle('BERT Base') 
g.savefig('imgs/A3_bertbase.svg', format='svg', bbox_inches='tight')

df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'bert-large-uncased']
df = df.loc[df['head'] >= 0]

df2 = get_results(dir_path='./ds_linear_head_wise_results')
df2 = df2.loc[df2['location'] == 'head']
df2 = df2.loc[df2['model_name'] == 'bert-large-uncased']
df2 = df2.loc[df2['head'] >= 0]
df = pd.concat([df, df2])
df = df.loc[df['task'] != 'MRPC']
df = df.pivot_table(index=['layer', 'head'], columns='task', values='acc')
df = df.reset_index().drop(columns=['head', 'layer'])
#df = df[task_order]

df = df.corr(method='spearman')

#sns.heatmap(df, annot=True, cmap='bwr', center=0)
g = sns.clustermap(df, cmap='bwr', figsize=(7,7), center=0)
g.fig.suptitle('BERT Large') 
g.savefig('imgs/A3_bertlarge.svg', format='svg', bbox_inches='tight')

df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'openai-gpt']
df = df.loc[df['head'] >= 0]

df2 = get_results(dir_path='./ds_linear_head_wise_results')
df2 = df2.loc[df2['location'] == 'head']
df2 = df2.loc[df2['model_name'] == 'openai-gpt']
df2 = df2.loc[df2['head'] >= 0]
df = pd.concat([df, df2])
df = df.loc[df['task'] != 'MRPC']

df = df.pivot_table(index=['layer', 'head'], columns='task', values='acc')
df = df.reset_index().drop(columns=['head', 'layer'])
#df = df[task_order]

df = df.corr(method='spearman')

#sns.heatmap(df, annot=True, cmap='bwr', center=0)
g = sns.clustermap(df, cmap='bwr', figsize=(7,7), center=0)
g.fig.suptitle('GPT') 
g.savefig('imgs/A3_gpt.svg', format='svg', bbox_inches='tight')

df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'gpt2']
df = df.loc[df['head'] >= 0]

df2 = get_results(dir_path='./ds_linear_head_wise_results')
df2 = df2.loc[df2['location'] == 'head']
df2 = df2.loc[df2['model_name'] == 'gpt2']
df2 = df2.loc[df2['head'] >= 0]
df = pd.concat([df, df2])
df = df.loc[df['task'] != 'MRPC']

df = df.pivot_table(index=['layer', 'head'], columns='task', values='acc')
df = df.reset_index().drop(columns=['head', 'layer'])
#df = df[task_order]

df = df.corr(method='spearman')

#sns.heatmap(df, annot=True, cmap='bwr', center=0)
g = sns.clustermap(df, cmap='bwr', figsize=(7,7), center=0)
g.fig.suptitle('GPT2') 
g.savefig('imgs/A3_gpt2.svg', format='svg', bbox_inches='tight')

# Head-wise Similarity Analysis

In [ ]:
from scipy.cluster.hierarchy import linkage

df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'bert-large-uncased']
df = df.loc[df['head'] >= 0]
df = df.pivot_table(index=['layer', 'head'], columns='task', values='acc')
df_layer = df.index.get_level_values('layer')
lut = dict(zip(df_layer.unique(), sns.color_palette("Blues", len(df_layer.unique()))))
col_colors = pd.Series(df_layer.values).map(lut)


df = df.reset_index().drop(columns=['head', 'layer'])
df = df[task_order]

# ====== Column-wise Normalization ===== #
df=(df-df.mean())/df.std() # std norm
# df=(df-df.min())/(df.max()-df.min())

Z = linkage(df.values, metric='euclidean')
df = df.T.corr(method=euclidean)
# display(df)
g = sns.clustermap(df, cmap='jet', metric='euclidean', standard_scale=1, row_cluster=True, col_cluster=True, col_colors=col_colors, row_colors=col_colors)
col_idx = g.dendrogram_col.reordered_ind
# sns.heatmap(df, cmap='bwr', center=0)

g = sns.clustermap(df, cmap='jet', metric='euclidean', standard_scale=1, row_cluster=False, col_cluster=False, col_colors=col_colors, row_colors=col_colors, row_linkage=Z, col_linkage=Z)



# Head-wise Dimensionality Reduction

In [ ]:
df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'bert-large-uncased']
df = df.pivot_table(index=['layer', 'head'], columns='task', values='acc')

df = df.reset_index()
df = df.loc[df['head'] >= 0]
layer = df['layer']
df = df.drop(columns=['head', 'layer'])

# ====== Column-wise Normalization ===== #
# df=(df-df.mean())/df.std() # std norm
# df=(df-df.min())/(df.max()-df.min())
# display(df)


# print(layer.values.shape)
X = df.values


X_embedded = TSNE(n_components=2).fit_transform(X)
# X_embedded = PCA(n_components=2).fit_transform(X)
# print(X_embedded.shape)
plt.scatter(X_embedded[:,0], X_embedded[:,1], c=layer.values)
plt.legend()

# Average Head Feature

In [ ]:
df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'bert-base-uncased']
df = df.loc[df['head'] >= 0]
df = df.loc[(df['layer'] == 0) | (df['layer'] == 11) | (df['layer'] == 4)]
df = df.pivot_table(index=['layer', 'head'], columns='task', values='acc')

df_avg = df.groupby(level=0).mean()
df = df.sub(df_avg, level=0)

df = df.reset_index() #.drop(columns=['head', 'layer'])
df_task = df[task_order]
df[task_order]=(df_task-df_task.mean())/df_task.std()
df['model_name'] = 'base'
# normalized_df
# display(df)

df2 = get_results(dir_path='./mlp_results')
df2 = df2.loc[df2['model_name'] == 'bert-large-uncased']
df2 = df2.loc[df2['head'] >= 0]
# df2 = df2.loc[(df2['layer'] == 0) | (df2['layer'] == 11) | (df2['layer'] == 4)]
df2 = df2.pivot_table(index=['layer', 'head'], columns='task', values='acc')

df_avg2 = df2.groupby(level=0).mean()
df2 = df2.sub(df_avg2, level=0)

df2 = df2.reset_index() #.drop(columns=['head', 'layer'])
df_task2 = df2[task_order]
df2[task_order]=(df_task2-df_task2.mean())/df_task2.std()
df2['model_name'] = 'large'


# display(df2)

df = df.append(df2)

sns.pairplot(df, hue='model_name', vars=['Length',  'WordContent', 'Depth', 'TopConstituents', 'BigramShift', 'Tense', 'SubjNumber', 'ObjNumber', 'OddManOut', 'CoordinationInversion'])

In [ ]:
df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'bert-large-uncased']
df = df.loc[df['head'] >= 0]

df = df.pivot_table(index=['layer', 'head'], columns='task', values='acc')
df_avg = df.groupby(level=0).mean()
df = df.sub(df_avg, level=0)

df = df.reset_index()
layer = df['layer']
df = df.drop(columns=['head', 'layer'])

df_task = df[task_order]
# df[task_order]=(df_task-df_task.mean())/df_task.std()
# display(df)

X = df.values
print(X.shape)
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
X_embedded = TSNE(n_components=2).fit_transform(X)
# X_embedded = PCA(n_components=2).fit_transform(X)
plt.scatter(X_embedded[:,0], X_embedded[:,1], c=layer.values)
plt.legend()

In [ ]:
df = get_results(dir_path='./linear_head_wise_results')
df = df.loc[df['model_name'] == 'bert-large-uncased']
df = df.loc[df['head'] >= 0]
df = df.pivot_table(index=['layer', 'head'], columns='task', values='acc')

df_avg = df.groupby(level=0).mean()
df = df.sub(df_avg, level=0)

df_layer = df.index.get_level_values('layer')
# print(df_layer.unique())
lut = dict(zip(df_layer.unique(), sns.color_palette("Blues", len(df_layer.unique()))))
col_colors = pd.Series(df_layer.values).map(lut)
# display(df)
# display(col_colors)

df = df.reset_index().drop(columns=['head', 'layer'])
df = df[task_order]

df = df.T.corr(method=euclidean)
# display(df)
sns.clustermap(df, cmap='jet', metric='euclidean', standard_scale=1, row_cluster=True, col_cluster=True, col_colors=col_colors, row_colors=col_colors)
# sns.heatmap(df, cmap='bwr', center=0)
